In [1]:
from owlready2 import *
import pandas as pd

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
onto = get_ontology("/Users/sarahreb/Desktop/local/protege_files/cds-ontology.owl").load()

gndo = Namespace(
    world_or_ontology=onto,
    base_iri="https://d-nb.info/gnd/",
    name="gndo"
)

cds_docs = Namespace(
    world_or_ontology=onto,
    base_iri="https://constance-de-salm.de/archiv/#/document/",
    name="cds_docs"
)

In [3]:
"""Individuals are instances in ontologies. They are created as any other Python instances. The first parameter is the name (or identifier) of the Individual; it corresponds to the .name attribute in Owlready2. If not given, the name if automatically generated from the Class name and a number.
"""

'Individuals are instances in ontologies. They are created as any other Python instances. The first parameter is the name (or identifier) of the Individual; it corresponds to the .name attribute in Owlready2. If not given, the name if automatically generated from the Class name and a number.\n'

In [4]:
# Parsing CSV file as DataFrame in order to create individuals.

df = pd.read_csv('../data/retrieved/filtered_cds_data.csv')
df = df.fillna(0)
df

,Unnamed: 0,FuD-Key,letzte Änderung,Dokumenttyp,URL,Nr.,Verfasser,GND (Verfasser),VIAF (Verfasser),Empfänger,...,Geonames (Orte),Schlagwörter,Editionen,Vermerk,Bestand,Verweise,Digitalisate: Dateiname (Signatur),Zitierempfehlung,year,decade
0,100,CdS-b1-000s,2022-08-19 10:18:12,Brief,https://constance-de-salm.de/archiv/#/document/28,C11/S84/020-022,"Salm, Constance de",http://d-nb.info/gnd/116765968,https://viaf.org/viaf/121051,"Barbier, Antoine-Alexandre",...,https://www.geonames.org/3247449; https://www....,Wohnen/Haushalt; Frankreich - Politik; CdS: Ép...,0,0,Toulon,0,CdS-b1-000s-0.jpg (C11/S84/020 ); CdS-b1-000s-...,C11/S84/020-022. In: Die Korrespondenz der Con...,1822.0,1820.0
1,108,CdS-b1-0021,2022-08-19 10:18:12,Brief,https://constance-de-salm.de/archiv/#/document/73,C11/S84/117-119,"Salm, Constance de",http://d-nb.info/gnd/116765968,https://viaf.org/viaf/121051,"Barbier, Antoine-Alexandre",...,https://www.geonames.org/2934246; https://www....,Briefpartner/Werk; Literarisches Geschehen/Neu...,0,0,Toulon,0,CdS-b1-0021-0.jpg (C11/S84/117 ); CdS-b1-0021-...,C11/S84/117-119. In: Die Korrespondenz der Con...,1823.0,1820.0
2,110,CdS-b1-000z,2022-08-19 10:18:12,Brief,https://constance-de-salm.de/archiv/#/document/35,C11/S84/037-038,"Salm, Constance de",http://d-nb.info/gnd/116765968,https://viaf.org/viaf/121051,"Barbier, Antoine-Alexandre",...,0,Karriere/-förderung,0,0,Toulon,0,CdS-b1-000z-0.jpg (C11/S84/037 ); CdS-b1-000z-...,C11/S84/037-038. In: Die Korrespondenz der Con...,1822.0,1820.0
3,111,CdS-b1-0026,2022-08-19 10:18:12,Brief,https://constance-de-salm.de/archiv/#/document/78,C11/S84/131,"Salm, Constance de",http://d-nb.info/gnd/116765968,https://viaf.org/viaf/121051,"Fallenstein, Georg Friedrich (?)",...,0,CdS: Épître à un honnête homme (1820); CdS: Su...,0,0,Toulon,0,CdS-b1-0026-0.jpg (C11/S84/131),C11/S84/131. In: Die Korrespondenz der Constan...,1823.0,1820.0
4,112,CdS-b1-0016,2022-08-19 10:18:12,Brief,https://constance-de-salm.de/archiv/#/document/42,C11/S84/052,"Salm, Constance de",http://d-nb.info/gnd/116765968,https://viaf.org/viaf/121051,"Preußen, Prinz Friedrich von (Neffe Friedrich ...",...,https://www.geonames.org/2934246,"CdS: Poésies (1811, 1817², 1835³)",0,0,Toulon,0,CdS-b1-0016-0.jpg (C11/S84/052),C11/S84/052. In: Die Korrespondenz der Constan...,1822.0,1820.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026,10716,CdS-b1-08iu,2022-08-19 10:18:12,Brief,https://constance-de-salm.de/archiv/#/document...,CdS/31/125-126,"Salm, Constance de",http://d-nb.info/gnd/116765968,https://viaf.org/viaf/121051,"Pailliet, Jean Baptiste Joseph (Anwalt der CdS...",...,https://www.geonames.org/2989317,Wirtschaft/Finanzen; Prozess/Gericht,0,0,Schloss Dyck,0,CdS-b1-08iu-0.jpg (CdS/31/125); CdS-b1-08iu-1....,CdS/31/125-126. In: Die Korrespondenz der Cons...,1829.0,1820.0
2027,10717,CdS-b1-08jb,2022-08-19 10:18:12,Brief,https://constance-de-salm.de/archiv/#/document...,CdS/31/098-099,"Salm, Constance de",http://d-nb.info/gnd/116765968,https://viaf.org/viaf/121051,"Pailliet, Jean Baptiste Joseph (Anwalt der CdS...",...,https://www.geonames.org/2972191,Prozess/Gericht,0,0,Schloss Dyck,0,CdS-b1-08jb-0.jpg (CdS/31/098); CdS-b1-08jb-1....,CdS/31/098-099. In: Die Korrespondenz der Cons...,1827.0,1820.0
2028,10720,CdS-b1-08in,2022-08-19 10:18:12,Brief,https://constance-de-salm.de/archiv/#/document...,CdS/31/039-040,"Salm, Constance de",http://d-nb.info/gnd/116765968,https://viaf.org/viaf/121051,"Pailliet, Jean Baptiste Joseph (Anwalt der CdS...",...,https://www.geonames.org/2972191; https://www....,Prozess/Gericht,0,0,Schloss Dyck,0,CdS-b1-08in-0.jpg (CdS/31/039); CdS-b1-08in-1....,CdS/31/039-040. In: Die Korrespondenz der Cons...,1825.0,1820.0
2029,10721,CdS-b1-08iz,2022-08-19 10:18:12,Brief,https://constance-de-salm.de/archiv/#/document...,CdS/31/062-063,"Salm, Constance de",http://d-nb.info/gnd/116765968,https://viaf.org/viaf/121051,"Pailliet, Jean Baptiste Joseph (Anwalt der CdS...",...,

In [6]:
# Adding instances of the CdS letters with senders and addressees.
index = 0
for url in df['URL']:
    new_letter = onto.Letter(re.sub('https://constance-de-salm.de/archiv/#/document/', '', url), namespace=cds_docs)
    new_letter.fud_key = df['FuD-Key'][index]
    new_letter.has_year.append(int(df['year'][index]))
    new_letter.has_decade.append(int(df['decade'][index]))
    new_letter.has_date.append(df['Datierung (JJJJ-MM-TT)'][index])
    new_sender = onto.Sender(re.sub('http://d-nb.info/gnd/', '', df['GND (Verfasser)'][index]), namespace=gndo)
    new_sender.label = df['Verfasser'][index]
    new_sender.has_viaf = df['VIAF (Verfasser)'][index]
    new_letter.has_sender.append(new_sender)
    if df['GND (Empfänger)'][index] == 0:
        new_addressee = onto.Sender(str(index), namespace=onto)
    else:
        new_addressee = onto.Sender(re.sub('http://d-nb.info/gnd/', '', df['GND (Empfänger)'][index]), namespace=gndo)
    new_addressee.label = df['Empfänger'][index]
    new_addressee.has_viaf = df['VIAF (Empfänger)'][index]
    new_letter.has_addressee.append(new_addressee)
    index += 1

In [8]:
for i in onto.Letter.instances():
    #destroy_entity(i)
    print(i, i.has_date, i.fud_key, i.has_decade, i.has_year, i.has_sender, i.has_addressee)

cds_docs.28 ['1822-04-25'] CdS-b1-000s [1820] [1822] [gndo.116765968] [gndo.116051795]
cds_docs.73 ['1823-08-15'] CdS-b1-0021 [1820] [1823] [gndo.116765968] [gndo.116051795]
cds_docs.35 ['1822-09-14'] CdS-b1-000z [1820] [1822] [gndo.116765968] [gndo.116051795]
cds_docs.78 ['1823-10-17'] CdS-b1-0026 [1820] [1823] [gndo.116765968] [gndo.11639692X]
cds_docs.42 ['1822-12-06'] CdS-b1-0016 [1820] [1822] [gndo.116765968] [cds-ontology.4]
cds_docs.47 [0] CdS-b1-001b [0] [0] [gndo.116765968] [gndo.1067648739]
cds_docs.237 ['1825-11-14'] CdS-b1-006l [1820] [1825] [gndo.116765968] [gndo.121734307]
cds_docs.251 [0] CdS-b1-006z [0] [0] [gndo.116765968] [gndo.121734307]
cds_docs.256 ['1829-12-08'] CdS-b1-0074 [1820] [1829] [gndo.116765968] [gndo.121734307]
cds_docs.249 ['1827-08-02'] CdS-b1-006x [1820] [1827] [gndo.116765968] [gndo.121734307]
cds_docs.489 [0] CdS-b1-00dj [0] [0] [gndo.116765968] [cds-ontology.10]
cds_docs.756 ['1819-01-25'] CdS-b1-00ju [1810] [1819] [gndo.116765968] [cds-ontology.11

In [9]:
onto.save(file = "cds_onto_test.owl", format = "rdfxml")